In [40]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import librosa

In [41]:
## Import files "written"
written_train = np.load("written_train.npy", allow_pickle=True)

# # Import files "spoken"
spoken_train = np.load('spoken_train.npy', allow_pickle=True)

# # Import file "match"
match_train = np.load('match_train.npy', allow_pickle=True)

written_corner = np.load('image_hoek.npy', allow_pickle=True)
plasma_sd = np.load('plasma_sd_Z.npy', allow_pickle=True)
plasma_m = np.load('plasma_mean_Z.npy', allow_pickle=True)

In [42]:
lengths_spoken = np.empty(len(spoken_train))
for index, value in enumerate(spoken_train):
    lengths_spoken[index] = value.shape[0]

sorted_lengths = np.sort(lengths_spoken)
longest_length = sorted_lengths[-1]
print(longest_length)

padden_spoken = []

for i in spoken_train:
    if i.shape[0] < longest_length:
        pad_size = longest_length - i.shape[0]
        length = i.shape[1]
        pad_array = np.zeros((13,), dtype=np.int)
        x = np.tile(pad_array, (int(pad_size),1))
        i = np.concatenate((i, x), axis=0)
        padden_spoken.append(i)
    else:
        padden_spoken.append(i)

93.0


In [43]:
nieuw = np.array(padden_spoken)
nieuw_2 = []
for i in range(0,len(nieuw)):
    temp_list = np.array(nieuw[i].flatten())
    nieuw_2.append(temp_list)

In [44]:
nieuw_3 = np.array(nieuw_2)

In [45]:
nieuw_4 = np.concatenate((nieuw_3, written_train), axis=1)

In [46]:
target = []
indexes_true = []
indexes_false = []
                       
for i, j in enumerate(match_train):
    if j == True:
        target.append(1)
        indexes_true.append(i)
    elif j == False:
        target.append(-1)
        indexes_false.append(i)
        
false_included = indexes_false[:6000]
included_rows = indexes_true + false_included

In [47]:
import pandas as pd
df_a=pd.DataFrame(data=nieuw_4[:,:],    # values
             index=range(0,nieuw_4.shape[0]),    # 1st column as index
             columns=range(0,nieuw_4.shape[1]))

In [49]:
# Split data into X's (summaries) and y (which is the target value)

X = df_a.iloc[included_rows, 0:1993].values
y = np.array(target)[included_rows]

In [50]:
y

array([ 1,  1,  1, ..., -1, -1, -1])

In [51]:

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [81]:
## Import the model we are using
#from sklearn.ensemble import RandomForestRegressor
#
## Instantiate model 
#rf = RandomForestRegressor(n_estimators= 1000, random_state=42)
#
## Train the model on training data
#rf.fit(X_train, y_train);

In [68]:
rf_new = RandomForestRegressor(n_estimators = 10, criterion = 'mse', max_depth = None, 
                               min_samples_split = 2, min_samples_leaf = 1)
rf_new.fit(X_train, y_train);

In [69]:
# Use the forest's predict method on the test data
predictions = rf_new.predict(X_val)

# Calculate the absolute errors
errors = abs(predictions - y_val)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


Mean Absolute Error: 0.78 degrees.


In [70]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_val)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 99.15 %.


In [71]:
count_true = 0
count_false = 0
for i in y_val:
    if i == 1:
        count_false += 1
    else:
        count_true += 1

In [87]:
print(count_false)

1215


In [72]:
count_true/(count_true+count_false)

0.5555028462998103

In [73]:
index = range(0,len(y_val))
columns = ['predictions', 'y_val']


df_b = pd.DataFrame(index=index, columns=columns)
df_b['predictions'] = predictions
df_b['y_val'] = y_val


df_b


,predictions,y_val
0,-0.4,-1
1,0.2,1
2,-0.2,1
3,-0.2,1
4,-0.2,-1
5,0.0,1
6,-0.6,-1
7,0.2,1
8,0.4,-1
9,0.6,1


In [77]:
y_pred = np.array(df_b["predictions"])

In [78]:
for index, value in enumerate(y_pred):
    if value >= 0:
        y_pred[index] = 1
    else: y_pred[index] = -1

In [79]:
y_pred

array([-1.,  1., -1., ...,  1., -1., -1.])

In [80]:
## TN, FP
## FN, TP
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
print(cm)

[[856 315]
 [276 661]]


In [66]:
#[[922 231]
# [480 475]]

In [49]:
[[893 284]
 [489 442]]

1153

In [50]:
count_false +count_true 

2108

In [51]:
922+231+480+475

2108